In [1]:
%reset
%matplotlib inline
from feature_engineering import *

#regression imports 
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor


#model selection
from sklearn.model_selection import train_test_split,cross_validate

# evaluation metrics
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score, mean_absolute_error 

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


/usr/lib/python3/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Import data

In [ ]:
# Read in data
data = pd.read_csv('./data/data.csv')

# what am I trying to predict 
predict_stations = True

In [ ]:
data['day'] = data['date'].apply(lambda x : int(x.split('-')[-1]))
train_df = data[data.day <= 19]
test_df = data[data.day > 19]

In [ ]:
del data

In [ ]:
if predict_stations == False:
    train_df = train_df.drop(['station', 'total_in'], axis = 1)
    train_df = train_df.groupby(['date', 'time', 'member_type', 
                               'weather_description', 'temp_in_f', 'humidity_in_%', 'pressure',
                               'visibility_in_miles', 'wind_direction', 'wind_speed_in_mph',
                               'temp_in_f_delta', 'pressure_delta', 'humidity_delta',
                               'visibility_delta', 'wind_speed_delta', 'day'], as_index = False)['total_out'].sum()

In [ ]:
train_df.columns

In [ ]:
train_df.head()

In [ ]:
train_df.visibility_in_miles.value_counts().values[1:].sum()

## Feature engineering

In [ ]:
# do feature engineering 
train_df = holiday(train_df)
train_df = season(train_df)
train_df = date_feature(train_df)
train_df = member_type(train_df)
train_df = weather(train_df)
train_df = wind_direction(train_df)
if predict_stations == True:
    train_df = top_i_station_onehot(train_df, top = 25)
    
    # keep only data for the top stations 
    train_df = train_df[train_df['other'] == 0]
    
train_df = hour_features(train_df)
train_df = humidity_feature(train_df)
train_df = visibility_feature(train_df)
train_df = heat_index(train_df)
train_df = wind_chill(train_df)

#test_df = holiday(test_df)
#test_df = season(test_df)
#test_df = date_feature(test_df)
#test_df = member_type(test_df)
#test_df = weather(test_df)
#test_df = wind_direction(test_df)

In [ ]:
train_df.columns

In [ ]:
## need to do something better with this
train_df['time'] =  train_df['time'].apply(lambda x: int(x.split(':')[0]))

## Models

In [ ]:
# Splitting train data into cross validation elements.
# Right now we predict total_in Problem label flow not correct 

drop_features = [ 'total_in', 'station', 'total_out', 'day', 'pressure_delta', 'pressure', 
                 'visibility_delta', 'Casual', 'sum_ins_outs', 'other']

models = [RandomForestRegressor(max_depth = 5, n_estimators = 100), AdaBoostRegressor(), 
          BaggingRegressor(), KNeighborsRegressor()]#  Lasso(alpha = 0.1, max_iter=100)] #SVR() MLPRegressor(random_state=0, activation='relu', hidden_layer_sizes=(2,))
model_names = ['RandomForest', 'AdaBoost', 'Bagging', 'Kneighnours'] #'Lasso'] # SVR 'Neural Network', 


if predict_stations == False:
    drop_features = [ 'total_out', 'day', 'visibility_delta', 'Casual'] 
    models = [RandomForestRegressor(bootstrap=True,
            max_depth=10, n_estimators=100, random_state=42), AdaBoostRegressor(), 
          BaggingRegressor(), KNeighborsRegressor()] #SVR(), , Lasso(alpha = 0.1, max_iter=100)
    model_names = ['RandomForest', 'AdaBoost', 'Bagging', 'Kneighnours'] # SVR

X_train, X_test, y_train, y_test = train_test_split(train_df.loc[train_df.Member == 1 ].drop(drop_features, axis = 1),
                                                    train_df.loc[train_df.Member == 1 , 'sum_ins_outs'], test_size = 0.20, random_state = 42)

# SVR removed for now cause f**

log_rmse = []
mse = []
r2 = []
mae = []
d = {}
for model in range (len(models)):
    print('Running model: ', model_names[model])
    clf = models[model]
    clf.fit(X_train, y_train)
    print('model fitted..')
    test_pred = clf.predict(X_test)
    print('predictions done..')
    model_log_rmse = np.sqrt(mean_squared_log_error(list(map(lambda x: x + 0.1, test_pred)), y_test))
    mse.append(mean_squared_error(test_pred, y_test))
    r2.append(r2_score(test_pred, y_test))
    mae.append(mean_absolute_error(test_pred, y_test))
    log_rmse.append(model_log_rmse)
    print(model_log_rmse)

d = {'Modelling Algo':model_names,'Log_RMSE':log_rmse, 'MSE':mse, 'R2':r2, 'MAE': mae} 

rmse_df = pd.DataFrame(d)
rmse_df

#### PREDICTIONS INCLUDING THE STATIONS 
Predicting sum ins and outs 

In [ ]:
train_df.shape